In [1]:
# default_exp tools

In [2]:
#export
import types
import functools
from nbdev.imports import *

# Notebook Tools

These tools are used to improve the notebook developing environment. Mo

## Class Patch Tools

In [3]:
#export
def copy_func(f):
    "Copy a non-builtin function (NB `copy.copy` does not work for this)"
    if not isinstance(f,types.FunctionType): return copy(f)
    fn = types.FunctionType(f.__code__, f.__globals__, f.__name__, f.__defaults__, f.__closure__)
    fn.__dict__.update(f.__dict__)
    return fn

In [4]:
#export
def patch_to(cls, as_prop=False):
    "Decorator: add `f` to `cls`"
    if not isinstance(cls, (tuple,list)): cls=(cls,)
    def _inner(f):
        for c_ in cls:
            nf = copy_func(f)
            # `functools.update_wrapper` when passing patched function to `Pipeline`, so we do it manually
            for o in functools.WRAPPER_ASSIGNMENTS: setattr(nf, o, getattr(f,o))
            nf.__qualname__ = f"{c_.__name__}.{f.__name__}"
            setattr(c_, f.__name__, property(nf) if as_prop else nf)
        return f
    return _inner

In [5]:
class _T1(int): pass

@patch_to(_T1)
def func1(x, a): return x+a

t = _T1(1)
test_eq(t.func1(2), 3)

In [6]:
class _T2(int): pass
@patch_to((_T1,_T2))
def func2(x, a): return x+2*a

t = _T1(1)
test_eq(t.func2(1), 3)
t = _T2(1)
test_eq(t.func2(1), 3)

In [7]:
#export
def patch(f):
    "Decorator: add `f` to the first parameter's class (based on f's type annotations)"
    cls = next(iter(f.__annotations__.values()))
    return patch_to(cls)(f)

In [8]:
@patch
def func3(x:_T1, a):
    "test"
    return x+2

t = _T1(1)
test_eq(t.func3(2), 3)
test_eq(t.func3(10), 3)
test_eq(t.func3.__qualname__, '_T1.func3')

In [9]:
@patch
def func4(x:(_T1,_T2), a):
    "test"
    return x+2*a

t = _T1(1)
test_eq(t.func4(2), 5)
test_eq(t.func4.__qualname__, '_T1.func4')
t = _T2(1)
test_eq(t.func4(2), 5)
test_eq(t.func4.__qualname__, '_T2.func4')

In [10]:
#export
def patch_property(f):
    "Decorator: add `f` as a property to the first parameter's class (based on f's type annotations)"
    cls = next(iter(f.__annotations__.values()))
    return patch_to(cls, as_prop=True)(f)

In [11]:
@patch_property
def prop(x:_T1): return x+1

t = _T1(1)
test_eq(t.prop, 2)

## Delegates

In [12]:
def test_sig(f, b): test_eq(str(inspect.signature(f)), b)

In [13]:
#export
def delegates(to=None, keep=False):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to,f
        from_f = getattr(from_f,'__func__',from_f)
        if hasattr(from_f,'__delwrap__'): return f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        from_f.__delwrap__ = to_f
        return f
    return _f

In [15]:
def _T3(e, c=2): pass

@delegates(basefoo)
def d(a, b=1, **kwargs): pass
test_sig(foo, '(a, b=1, c=2)')

@delegates(basefoo, keep=True)
def foo(a, b=1, **kwargs): pass
test_sig(foo, '(a, b=1, c=2, **kwargs)')